In [1]:
import numpy as np
import pandas as pd

In [2]:
data_url = "http://users.stat.ufl.edu/~winner/data/nfl2008_fga.csv"

In [3]:
df = pd.read_csv("C:/datasets/NFL/nfl2008_fga.csv")

In [4]:
df.head()

,GameDate,AwayTeam,HomeTeam,qtr,min,sec,kickteam,def,down,togo,...,distance,homekick,kickdiff,timerem,offscore,defscore,season,GOOD,Missed,Blocked
0,20081130,IND,CLE,1,47,2,IND,CLE,4.0,11.0,...,30,0,-3,2822,0,3,2008,1,0,0
1,20081005,IND,HOU,1,54,47,IND,HOU,4.0,3.0,...,46,0,0,3287,0,0,2008,1,0,0
2,20081228,TEN,IND,1,45,20,IND,TEN,4.0,3.0,...,28,1,7,2720,7,0,2008,1,0,0
3,20081012,BAL,IND,1,45,42,IND,BAL,4.0,1.0,...,37,1,14,2742,14,0,2008,1,0,0
4,20080907,CHI,IND,1,50,56,IND,CHI,4.0,21.0,...,39,1,0,3056,0,0,2008,1,0,0


In [5]:
df['timeremhalf'] = df.timerem %(60 * 30)
df['overtime'] = (df.qtr == 5)+0
df['twominutewarning'] = (((df.qtr == 4)|(df.qtr == 2))&(df.timeremhalf <= 120))+0

In [6]:
df.describe()

,GameDate,qtr,min,sec,down,togo,kicker,ydline,distance,homekick,...,timerem,offscore,defscore,season,GOOD,Missed,Blocked,timeremhalf,overtime,twominutewarning
count,1.039000e+03,1039.000000,1039.000000,1039.000000,1037.000000,1037.000000,1039.000000,1039.000000,1039.000000,1039.000000,...,1039.000000,1039.000000,1039.000000,1039.0,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000
mean,2.008140e+07,2.497594,27.968239,26.551492,3.818708,6.738669,19.537055,18.768046,36.666025,0.494706,...,1704.645813,9.949952,9.567854,2008.0,0.866218,0.133782,0.021174,706.763234,0.012512,0.211742
std,1.702338e+03,1.108621,16.716783,18.107977,0.635134,4.155772,11.303244,10.187505,9.847836,0.500213,...,1003.771275,9.434869,8.365389,0.0,0.340582,0.340582,0.144034,524.990531,0.111209,0.408740
min,2.008090e+07,1.000000,-15.000000,0.000000,1.000000,1.000000,1.000000,1.000000,18.000000,0.000000,...,-887.000000,0.000000,0.000000,2008.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.008100e+07,2.000000,14.000000,9.500000,4.000000,4.000000,9.000000,10.000000,28.000000,0.000000,...,895.500000,0.000000,3.000000,2008.0,1.000000,0.000000,0.000000,188.500000,0.000000,0.000000
50%,2.008111e+07,2.000000,30.000000,25.000000,4.000000,6.000000,20.000000,19.000000,37.000000,0.000000,...,1808.000000,7.000000,7.000000,2008.0,1.000000,0.000000,0.000000,699.000000,0.000000,0.000000
75%,2.008120e+07,4.000000,42.000000,43.000000,4.000000,9.000000,30.000000,26.000000,44.000000,1.000000,...,2556.500000,16.000000,14.000000,2008.0,1.000000,0.000000,0.000000,1150.000000,0.000000,0.000000
max,2.009020e+07,5.000000,58.000000,59.000000,4.000000,25.000000,37.000000,86.000000,76.000000,1.000000,...,3507.000000,48.000000,44.000000,2008.0,1.000000,1.000000,1.000000,1737.000000,1.000000,1.000000


In [7]:
for col in df.columns.values:
    if df[col].isna().any():
        print(col)


down
togo


In [8]:
df.loc[df.down.isna() | df.togo.isna()]

,GameDate,AwayTeam,HomeTeam,qtr,min,sec,kickteam,def,down,togo,...,timerem,offscore,defscore,season,GOOD,Missed,Blocked,timeremhalf,overtime,twominutewarning
394,20081228,DET,GB,2,30,7,GB,DET,NaN,NaN,...,1807,14,7,2008,0,1,0,7,0,1
459,20081123,NYG,ARI,2,30,11,ARI,NYG,NaN,NaN,...,1811,12,17,2008,0,1,0,11,0,1


In [9]:
df = df.dropna()
df = df.drop(columns='season')

In [10]:
df.GameDate = df.GameDate.astype(int)
df['year'] = df.GameDate // 10000
df['month'] = (df.GameDate % 10000) // 100
df['day'] = df.GameDate % 100
df.GameDate = pd.to_datetime(df[['month','day','year']])

In [11]:
print(df.loc[df.GOOD + df.Missed + df.Blocked != 1,'GOOD':'Blocked'].describe())

       GOOD  Missed  Blocked
count  22.0    22.0     22.0
mean    1.0     0.0      1.0
std     0.0     0.0      0.0
min     1.0     0.0      1.0
25%     1.0     0.0      1.0
50%     1.0     0.0      1.0
75%     1.0     0.0      1.0
max     1.0     0.0      1.0


In [12]:
df[['GOOD','Blocked','Missed']].assign(count = 1).groupby(by=['GOOD','Blocked','Missed']).count().reset_index()

,GOOD,Blocked,Missed,count
0,0,0,1,137
1,1,0,0,878
2,1,1,0,22


In [13]:
df.GOOD -= df.Blocked

In [14]:
df[['GOOD','Blocked','Missed']].assign(count = 1).groupby(by=['GOOD','Blocked','Missed']).count().reset_index()

,GOOD,Blocked,Missed,count
0,0,0,1,137
1,0,1,0,22
2,1,0,0,878


In [15]:
df['result'] = df.GOOD * 2 + df.Missed
df.result = df.result.astype(int)

In [16]:
print(df.result.value_counts())

2    878
1    137
0     22
Name: result, dtype: int64


In [17]:
print(df.columns.values)

['GameDate' 'AwayTeam' 'HomeTeam' 'qtr' 'min' 'sec' 'kickteam' 'def'
 'down' 'togo' 'kicker' 'ydline' 'name' 'distance' 'homekick' 'kickdiff'
 'timerem' 'offscore' 'defscore' 'GOOD' 'Missed' 'Blocked' 'timeremhalf'
 'overtime' 'twominutewarning' 'year' 'month' 'day' 'result']


In [18]:
X = df.drop(columns = ['GameDate', 'AwayTeam', 'HomeTeam', 'kickteam', 'def', 'kicker', 'name','GOOD','Missed','Blocked','year','month','day','min','sec','ydline','result'])
X.describe()

,qtr,down,togo,distance,homekick,kickdiff,timerem,offscore,defscore,timeremhalf,overtime,twominutewarning
count,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000
mean,2.498554,3.818708,6.738669,36.604629,0.493732,0.380906,1704.444552,9.944069,9.563163,708.108968,0.012536,0.210222
std,1.109475,0.635134,4.155772,9.757380,0.500202,9.680840,1004.729213,9.442917,8.369894,524.600231,0.111315,0.407663
min,1.000000,1.000000,1.000000,18.000000,0.000000,-41.000000,-887.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,4.000000,4.000000,28.000000,0.000000,-6.000000,895.000000,0.000000,3.000000,191.000000,0.000000,0.000000
50%,2.000000,4.000000,6.000000,37.000000,0.000000,0.000000,1808.000000,7.000000,7.000000,700.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,44.000000,1.000000,6.000000,2557.000000,16.000000,14.000000,1152.000000,0.000000,0.000000
max,5.000000,4.000000,25.000000,76.000000,1.000000,44.000000,3507.000000,48.000000,44.000000,1737.000000,1.000000,1.000000


In [19]:
y = df[['GOOD','Missed','Blocked','result']]
y.describe()

,GOOD,Missed,Blocked,result
count,1037.000000,1037.000000,1037.000000,1037.000000
mean,0.846673,0.132112,0.021215,1.825458
std,0.360476,0.338776,0.144170,0.432073
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,2.000000
50%,1.000000,0.000000,0.000000,2.000000
75%,1.000000,0.000000,0.000000,2.000000
max,1.000000,1.000000,1.000000,2.000000


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [21]:
print(X_train.shape)
print(y_train.shape)

(829, 12)
(829, 4)


In [22]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver = 'lbfgs', max_iter=1000)
logistic.fit(X_train,y_train.GOOD.values.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
logistic.score(X_test,y_test.GOOD.values.ravel())

0.8028846153846154

In [24]:
logistic.score(X_train,y_train.GOOD.values.ravel())

0.8600723763570567

In [25]:
pd.DataFrame(
    {"attribute" : X.columns.values,
     "coefficient" : logistic.coef_[0]}
)

,attribute,coefficient
0,qtr,0.946683
1,down,0.231034
2,togo,-0.006213
3,distance,-0.102435
4,homekick,-0.148957
5,kickdiff,-0.002302
6,timerem,0.001261
7,offscore,0.019509
8,defscore,0.021811
9,timeremhalf,-0.000159
